In [62]:
import numpy as np
import pandas as pd
from datetime import datetime
import plotly
import plotly.express as px
import plotly.graph_objs as go
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [52]:
def read_user_data(userno):
    filename=f"data/User{userno}/User{userno}_behacom.csv"
    print(f'[INFO] reading file <{filename}>...')

    data_user=pd.read_csv(filename, encoding='latin-1',chunksize=1000)

    a = next(data_user)
    dtypes_col = a.dtypes.index
    dtypes_type = [i.name for i in a.dtypes.values]
    column_types = dict(zip(dtypes_col, dtypes_type))

    for k,v in column_types.items():
        if k == 'timestamp':
            column_types[k] = 'float64'
        elif ('average' in k):
            column_types[k] = 'float32'
        elif ('stddev' in k):
            column_types[k] = 'float32'
        elif v == 'float64':
            column_types[k] = 'float32'
        elif v == 'int64':
            if (k.startswith('press') or ('counter' in k) or ('usage' in k)):
                column_types[k] = 'int8'
            else:
                column_types[k] = 'int32'

    data_user=pd.read_csv(filename, encoding='latin-1',dtype=column_types)
    #print(data_user.shape)
    return data_user

In [53]:
total_users = 12
df_users = []
basic_info = { 'name':[], 'length':[]}
for i in range(total_users):
    df = read_user_data(i)
    df_users.append(df)
    basic_info['name'].append(f'User {i}')
    basic_info['length'].append(df.shape[0])

(6059, 12051)
(42281, 12051)
(179, 12051)
(3221, 12051)
(10114, 12051)
(2128, 12051)
(1332, 12051)
(55778, 12051)
(5404, 12051)
(7920, 12051)
(15358, 12051)
(17284, 12051)


NameError: name 'dt_group' is not defined

In [69]:
basic_info = pd.DataFrame(basic_info)
fig = px.bar(basic_info, x='name', y='length', color='name', title='Distribution of data per user')
fig.show()

In [56]:
# convert timestamp to datetime and add add as column
for i in range(total_users):
    df_users[i]['date_time'] = df_users[i][:]['timestamp'].astype('datetime64[ms]')

In [89]:
fig = go.Figure()
for i in range(total_users):
    df_grp_keystroke = df_users[i].groupby(pd.Grouper(key='date_time',freq='D')).agg({"keystroke_counter": "sum"}).reset_index().sort_values(by='date_time')
    fig.add_trace(go.Scatter(x=df_grp_keystroke['date_time'], y=df_grp_keystroke['keystroke_counter'], name=f'User {i}'))

fig.update_layout(
    title='Keystroke trend per user'
)
fig.show()

In [90]:
fig = go.Figure()
for i in range(total_users):
    df_grp_input_trend = df_users[i].groupby(pd.Grouper(key='date_time',freq='D')).size().to_frame(name='counts').reset_index().sort_values(by='date_time')
    fig.add_trace(go.Scatter(x=df_grp_input_trend['date_time'], y=df_grp_input_trend['counts'], name=f'User {i}'))

fig.update_layout(
    title='Input distribution over time per user'
)
fig.show()